In [ ]:
!pip install gymnasium
import gymnasium as gym
import math

import csv
import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
resultsFile='/content/drive/MyDrive/Results/BlackJackLlmRandom.csv'
overWriteResultsFile = True

In [ ]:

prompts = {
    "randomBaseLine": ""
}

In [ ]:
temperatures = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def generate(question, temperature=0.7, max_new_tokens=512):
    return 'RANDOM'

In [ ]:
def last_word_used(text):
    # Find the last index of "hit" and "right"
    hit_index = text.rfind("hit")
    stick_index = text.rfind("stick")

    # Compare the indices to determine the last word used
    if hit_index > stick_index:
        return "hit"
    elif stick_index > hit_index:
        return "stick"
    else:
        # Both words have the same index, handle this case as needed
        return "Both words appear at the same position"

In [ ]:
def insertValuesInPrompt(prompt, player_sum, dealer_card, ace):
    prompt = prompt.replace("#player_sum", str(player_sum))
    prompt = prompt.replace("#dealer_card", str(dealer_card))
    if(ace):
      prompt = prompt.replace("#ace_card", "You have a usable ace.")
    else:
      prompt = prompt.replace("#ace_card", "")

    #print(prompt)
    return prompt

In [ ]:
def runExperiment(episodes,promptKey, temperature):
    env = gym.make('Blackjack-v1')#, render_mode="human")
    prompt = prompts[promptKey]
    with open(resultsFile, 'a', newline='') as csvfile:
      # Create a CSV writer object
      csvwriter = csv.writer(csvfile)
      for i_episode in range(episodes):
          observation = env.reset()
          rad = observation[0][2]
          rewards = 0
          randomActions = 0
          hitActions = 0
          stickActions = 0

          player_sum = observation[0][0]
          dealer_card = observation[0][1]
          usable_ace = observation[0][2]

          for t in range(250):
              env.render()

              output = generate(insertValuesInPrompt(prompt, player_sum, dealer_card,usable_ace), temperature, 150)

              action = 0
              #print("OUTPUT")
              #print(output)
              #print("ENDOUTPUT")
              if last_word_used(output.lower()) == "stick":
                  action = 0
                  actionText = 'stick'
                  stickActions+=1
              elif last_word_used(output.lower()) == "hit":
                  action = 1
                  actionText = 'hit'
                  hitActions+=1
              else:
                  actionText = 'random'
                  randomActions+=1
                  action = env.action_space.sample() #Inconclusive answer, random action
              #print(output)
              #print(str(degrees) + " " + direction + " answer: "+actionText+" action: " + str(action))
              observation, reward, terminated, truncated, info = env.step(action)
              rad = observation[2]
              rewards += reward

              player_sum = observation[0]
              dealer_card = observation[1]
              usable_ace = observation[2]

              if terminated:
                  print('prompt: {}, temperature:{}, timesteps: {}, total rewards: {}, hit: {}, stick: {}, random: {}'.format(promptKey, temperature, t+1, rewards,hitActions,stickActions,randomActions))
                  if(overWriteResultsFile):
                    csvwriter.writerow([promptKey, temperature, t+1, rewards, hitActions, stickActions, randomActions,datetime.datetime.now()])
                  break

      env.close()


In [ ]:
#Create csv file
def createNewResultsCsv():
    if(overWriteResultsFile):
        with open(resultsFile, 'w', newline='') as csvfile:
            # Create a CSV writer object
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["promptKey", "temperature", "timesteps", "rewards", "hitActions", "stickActions", "randomActions", "DateTime"])

In [ ]:
def runExperiments(episodes,prompts, temperatures):
  createNewResultsCsv()
  for promptKey in prompts.keys():
    for temperature in temperatures:
      runExperiment(episodes, promptKey, temperature)

In [ ]:
runExperiments(100,prompts, temperatures)

prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: 1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: 1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 2, total rewards: -1.0, hit: 0, stick: 0, random: 2
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, stick: 0, random: 1
prompt: randomBaseLine, temperature:0, timesteps: 1, total rewards: -1.0, hit: 0, st

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/toy_text/blackjack.py:229: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("Blackjack-v1", render_mode="rgb_array")
  gym.logger.warn(
